In [1]:
import json
from bs4 import BeautifulSoup
import os
import pandas as pd
import spacy
import re
import numpy as np
import csv

C:\Users\Christina\anaconda3\envs\chatGPT\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def extract_content(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')

    index_entries = []

    primaryies = soup.find_all('div', class_='primaryie')

    for primaryie in primaryies:
        
        span = primaryie.find('span', class_='font-style-bold')
        
        if span:
            primaryie_text_tmp = primaryie.get_text().strip()
            primaryie_text_tmp = primaryie_text_tmp.split("\n")
            primaryie_text = primaryie_text_tmp[0]
            print(primaryie_text)
            
            
            
            secondaryies = primaryie.find_all('div', class_='secondaryie')
            if secondaryies:
                for secondaryie in secondaryies:
                    secondaryies_text = secondaryie.get_text().strip().split("\n")
                    secondaryies_text = secondaryies_text[0]                    
                    print("\t",secondaryies_text)
                    
                    tertiaryie_1 = secondaryie.find_all('div', class_='tertiaryie-1')
                    tertiaryie_2 = secondaryie.find_all('div', class_='tertiaryie-2')
                    tertiaryie_text=[]
                    if tertiaryie_1 or tertiaryie_2:
                        for item in tertiaryie_1:
                            tertiaryie_text.append(item.get_text())
                        for item in tertiaryie_2:
                            text = item.get_text()
                            tertiaryie_text.append(text)
                        
                        for item in tertiaryie_text:
                            print("\t\t ",item)
                        
                        primaryie_secondary = {
                             'primaryie': primaryie_text,
                             'secondaryies': secondaryies_text,
                             'tertiaryies': tertiaryie_text
                         }
                        
                        
                    else:
                        primaryie_secondary = {
                             'primaryie': primaryie_text,
                             'secondaryies': secondaryies_text
                         }
            
        
                    index_entries.append(primaryie_secondary)

    return index_entries

In [3]:
file_path = ('../index_book.html')
with open(file_path, 'r', encoding='utf-8') as file:
    html_content = file.read()
    index_entries = extract_content(html_content)

Abortion
	 Malaysia, [1.045]
	 United States, [1.041]
Acceptance, [4.052]–[4.063]
	 absolute and unqualified, [4.054]
		  case example, [4.055]
	 acceptor’s advantages, [4.119]
	 characteristics, [4.053]
	 communication of, [4.069]–[4.086]
		  completion, [4.077]
		  conditions unfulfilled despite, [4.075]
		  deeming, [4.071]
		  effective, determination, [4.077]
		  English law, [4.069], [4.070], [4.073]
		  general rule, [4.069]
		  Malaysia, [4.070]
		  means, [4.072]
		  omission, acceptance by, [4.072], [4.074]
		  silence, by, [4.073], [4.076]
		  timing, [4.078]
		  unilateral proposals, [4.075]
		  case, [4.079]
		  case analysis, [4.080], [4.081]
	 Contracts Act 1950 s 2(b), [4.052]
	 examples of analysis of existence, [4.006], [4.007]
	 mala fide, [4.098]
	 motive, [4.051]
	 necessity, [4.052]
	 omission, by, [4.072], [4.074]
	 oral contract case example, [4.057]
		  evidence, importance, [4.058]
	 postal acceptance, problems, [4.062], [4.063], [4.082]
		  case example, [4.0

In [4]:
df = pd.DataFrame(index_entries)

In [5]:
df.to_csv('all_index.csv',index=False, encoding='utf-8-sig')
df

,primaryie,secondaryies,tertiaryies
0,Abortion,"Malaysia, [1.045]",NaN
1,Abortion,"United States, [1.041]",NaN
2,"Acceptance, [4.052]–[4.063]","absolute and unqualified, [4.054]","[case example, [4.055]]"
3,"Acceptance, [4.052]–[4.063]","acceptor’s advantages, [4.119]",NaN
4,"Acceptance, [4.052]–[4.063]","characteristics, [4.053]",NaN
...,...,...,...
878,Yang Di-Pertuan Agong,"Head of Islam (own State, Penang and Melaka), ...",NaN
879,Yang Di-Pertuan Agong,"Head of State, [2.020]",NaN
880,Yang Di-Pertuan Agong,"part of tripartite legislative body, [2.019]",NaN
881,Yang Di-Pertuan Agong,"powers and restrictions, [2.020]",NaN


In [10]:
# Create new lists for the new DataFrame
new_data = {'ID': [], 'primaryie': [], 'secondaryies': [], 'tertiaryies': []}
id_count = 1
# Iterate through the rows of the original DataFrame
for index, row in df.iterrows():
    primary_col = row['primaryie']
    secondary_col = row['secondaryies']
    tertiary_col = row['tertiaryies']
    
    # Check if the tertiary_col is a list
    if isinstance(tertiary_col, list):
        # Iterate through the tertiary list in the current row
        for tertiary_item in tertiary_col:
            # Extract the actual tertiary value
            tertiary_value = tertiary_item

            # Append the values to the new_data dictionary
            new_data['ID'].append(id_count)
            new_data['primaryie'].append(primary_col)
            new_data['secondaryies'].append(secondary_col)
            new_data['tertiaryies'].append(tertiary_value)
            id_count +=1 
    else:
            new_data['ID'].append(id_count)
            new_data['primaryie'].append(primary_col)
            new_data['secondaryies'].append(secondary_col)
            new_data['tertiaryies'].append(tertiary_col)
            id_count +=1 
# Create a new DataFrame from the new_data dictionary
new_df = pd.DataFrame(new_data)


In [11]:
new_df.to_csv('all_index_byTertiary.csv',index=False, encoding='utf-8-sig')
new_df

,ID,primaryie,secondaryies,tertiaryies
0,1,Abortion,"Malaysia, [1.045]",NaN
1,2,Abortion,"United States, [1.041]",NaN
2,3,"Acceptance, [4.052]–[4.063]","absolute and unqualified, [4.054]","case example, [4.055]"
3,4,"Acceptance, [4.052]–[4.063]","acceptor’s advantages, [4.119]",NaN
4,5,"Acceptance, [4.052]–[4.063]","characteristics, [4.053]",NaN
...,...,...,...,...
1311,1312,Yang Di-Pertuan Agong,"Head of Islam (own State, Penang and Melaka), ...",NaN
1312,1313,Yang Di-Pertuan Agong,"Head of State, [2.020]",NaN
1313,1314,Yang Di-Pertuan Agong,"part of tripartite legislative body, [2.019]",NaN
1314,1315,Yang Di-Pertuan Agong,"powers and restrictions, [2.020]",NaN


In [128]:
def get_all_page_hyphen(text):
    paraNum = set()
    number_ranges = re.findall(r'\[(\d+\.\d+)\]–\[(\d+\.\d+)\]', text)
    for start, end in number_ranges:
            start_num = float(start)
            end_num = float(end)

            # Calculate the number of steps based on the desired precision
            precision = 0.001  # You can adjust this value
            num_steps = int((end_num - start_num) / precision) + 1

            # Generate numbers within the range with calculated number of steps
            numbers = np.linspace(start_num, end_num, num_steps)
            paraNum.update([f'{num:.3f}' for num in numbers])
    paraNum =sorted(paraNum)
    #print("paraNum ",paraNum)
    return paraNum

def get_all_page(text):
    number_ranges = re.findall(r'\[(\d+\.\d+)\]', text)
    return number_ranges

def extract_patterns(text):
    #print("input text : ",text)
    words = text
    page= set()
    #print(type(text))
    if type(text) != float:
        if "[" not in text:
            
            page.add(None)


        else:
            text = text.split(",")
            words = text[0]        
            for tx in text:

                if "[" in tx:
                    if "–" in tx:
                        page.update(get_all_page_hyphen(tx))
                    else:
                        tx = tx.strip()
                        #print("tx ",tx)
                        page.add(tx)
    page = sorted(page)
    #print("words: ",words)  
    #print("page: ",page) 
    return words,page

def get_mergewords(words_primary,words_secondary,words_tertiary):
        # Merge the columns
    merged_text = "{"
    if words_primary is not None:
        merged_text += words_primary
    if words_secondary is not None and type(words_secondary)!=float:
        merged_text += ":" + words_secondary
    if words_tertiary is not None and type(words_tertiary)!=float:
        merged_text += ":" + words_tertiary
    merged_text += "}"
    print(merged_text)
    return merged_text

In [129]:




new_data_2 = {'ID': [], 'primaryie_text': [], 'secondaryies_text': [], 'tertiaryies_text': [],'merged_text': [],'page_nums': []}  
        
for index, row in new_df.iterrows():
    id_count =  row['ID']
    primary_col = row['primaryie']
    secondary_col = row['secondaryies']
    tertiary_col = row['tertiaryies']
    
    merged_pages  = set()
    
    words_primary,page_primary = extract_patterns(primary_col)
    words_secondary,page_secondary = extract_patterns(secondary_col)
    words_tertiary,page_tertiary = extract_patterns(tertiary_col)
    
    merged_pages.update(page for page in page_primary if page is not None)
    merged_pages.update(page for page in page_secondary if page is not None)
    merged_pages.update(page for page in page_tertiary if page is not None)

    merged_pages =  sorted(merged_pages)
    
    
    merge_words = get_mergewords(words_primary,words_secondary,words_tertiary)
    
    # Append the values to the new_data dictionary
    new_data_2['ID'].append(id_count)
    new_data_2['primaryie_text'].append(words_primary)
    new_data_2['secondaryies_text'].append(words_secondary)
    new_data_2['tertiaryies_text'].append(words_tertiary)
    new_data_2['merged_text'].append(merge_words)
    new_data_2['page_nums'].append(merged_pages)
new_df2 = pd.DataFrame(new_data_2)
new_df2.to_csv('all_index_byTertiary.csv',index=False, encoding='utf-8-sig')

new_df2

{Abortion:Malaysia}
{Abortion:United States}
{Acceptance:absolute and unqualified:case example}
{Acceptance:acceptor’s advantages}
{Acceptance:characteristics}
{Acceptance:communication of:completion}
{Acceptance:communication of:conditions unfulfilled despite}
{Acceptance:communication of:deeming}
{Acceptance:communication of:effective}
{Acceptance:communication of:English law}
{Acceptance:communication of:general rule}
{Acceptance:communication of:Malaysia}
{Acceptance:communication of:means}
{Acceptance:communication of:omission}
{Acceptance:communication of:silence}
{Acceptance:communication of:timing}
{Acceptance:communication of:unilateral proposals}
{Acceptance:communication of:case}
{Acceptance:communication of:case analysis}
{Acceptance:Contracts Act 1950 s 2(b)}
{Acceptance:examples of analysis of existence}
{Acceptance:mala fide}
{Acceptance:motive}
{Acceptance:necessity}
{Acceptance:omission}
{Acceptance:oral contract case example:evidence}
{Acceptance:postal acceptance:cas

PermissionError: [Errno 13] Permission denied: 'all_index_byTertiary.csv'

In [130]:
# Function to handle various list formats
def flatten_and_convert(nums):
    flattened = []
    for num in nums:
        if isinstance(num, list):
            flattened.extend(num)
        else:
            flattened.append(num)
    return [convert_to_float(num) for num in flattened]

# Function to convert strings to floats and handle possible brackets
def convert_to_float(num):
    num = num.strip('[]')
    try:
        return float(num)
    except (ValueError, TypeError):
        return None

# Convert strings to floats, handling possible single-element lists and string brackets
new_df2["page_nums_converted"] = new_df2["page_nums"].apply(flatten_and_convert)

In [131]:
# Filter rows based on the range of page_nums_converted


filtered_df = new_df2[new_df2["page_nums_converted"].apply(lambda nums: any(4 <= num <= 8 for num in nums if num is not None))]


filtered_df.to_csv('all_index_filtered_df.csv',index=False, encoding='utf-8-sig')

filtered_df

,ID,primaryie_text,secondaryies_text,tertiaryies_text,merged_text,page_nums,page_nums_converted
2,3,Acceptance,absolute and unqualified,case example,{Acceptance:absolute and unqualified:case exam...,"[4.052, 4.053, 4.054, 4.055, 4.056, 4.057, 4.0...","[4.052, 4.053, 4.054, 4.055, 4.056, 4.057, 4.0..."
3,4,Acceptance,acceptor’s advantages,NaN,{Acceptance:acceptor’s advantages},"[4.052, 4.053, 4.054, 4.055, 4.056, 4.057, 4.0...","[4.052, 4.053, 4.054, 4.055, 4.056, 4.057, 4.0..."
4,5,Acceptance,characteristics,NaN,{Acceptance:characteristics},"[4.052, 4.053, 4.054, 4.055, 4.056, 4.057, 4.0...","[4.052, 4.053, 4.054, 4.055, 4.056, 4.057, 4.0..."
5,6,Acceptance,communication of,completion,{Acceptance:communication of:completion},"[4.052, 4.053, 4.054, 4.055, 4.056, 4.057, 4.0...","[4.052, 4.053, 4.054, 4.055, 4.056, 4.057, 4.0..."
6,7,Acceptance,communication of,conditions unfulfilled despite,{Acceptance:communication of:conditions unfulf...,"[4.052, 4.053, 4.054, 4.055, 4.056, 4.057, 4.0...","[4.052, 4.053, 4.054, 4.055, 4.056, 4.057, 4.0..."
...,...,...,...,...,...,...,...
1267,1268,Words and phrases,consideration,NaN,{Words and phrases:consideration},"[5.004, 5.005, 5.006, 5.007, 5.008, 5.009]","[5.004, 5.005, 5.006, 5.007, 5.008, 5.009]"
1280,1281,Words and phrases,invitation to treat,NaN,{Words and phrases:invitation to treat},[[4.024]],[4.024]
1284,1285,Words and phrases,offer,NaN,{Words and phrases:offer},[[4.014]],[4.014]
1293,1294,Words and phrases,sale by auction,NaN,{Words and phrases:sale by auction},[[4.034]],[4.034]


In [132]:
# Extract the column values as a list of lists
list_of_pages = filtered_df['page_nums_converted'].tolist()

# Convert the list of lists to a single set
merged_set = set(item for sublist in list_of_pages for item in sublist)

# Print or do something with the merged set
print("Unique paragraph numbers ")
print(merged_set)

Unique paragraph numbers 
{4.117, 4.087, 4.099, 4.056, 4.055, 4.052, 4.053, 4.054, 4.057, 4.058, 5.01, 7.032, 7.035, 16.017, 16.018, 4.061, 4.119, 4.07, 4.073, 4.006, 4.064, 4.122, 4.067, 4.125, 4.116, 5.006, 5.073, 5.058, 5.061, 5.055, 7.055, 7.052, 7.006, 4.025, 4.024, 7.003, 4.023, 4.017, 6.033, 4.016, 6.032, 4.015, 4.009, 4.097, 4.008, 4.098, 4.043, 4.031, 4.034, 4.113, 5.031, 5.034, 4.104, 4.107, 5.049, 6.028, 5.028, 5.037, 5.04, 5.043, 7.037, 7.04, 7.043, 7.049, 5.074, 4.126, 7.031, 7.034, 6.023, 7.046, 5.009, 6.016, 5.067, 6.009, 5.066, 4.11, 6.007, 4.05, 4.068, 4.071, 4.074, 4.077, 4.08, 4.083, 4.086, 5.013, 4.007, 5.004, 6.025, 5.007, 5.025, 5.022, 5.019, 7.007, 7.01, 7.022, 7.025, 4.042, 7.004, 4.101, 4.1, 4.094, 4.093, 4.027, 4.092, 4.001, 4.026, 4.059, 4.12, 4.065, 4.044, 4.062, 4.108, 5.059, 5.044, 5.047, 5.053, 5.056, 5.062, 5.065, 5.05, 4.019, 7.053, 7.044, 7.047, 7.05, 6.047, 4.018, 4.039, 4.011, 6.006, 4.01, 4.003, 6.019, 5.07, 4.127, 4.032, 4.084, 4.035, 4.029, 4.105,

In [133]:
filtered_df

,ID,primaryie_text,secondaryies_text,tertiaryies_text,merged_text,page_nums,page_nums_converted
2,3,Acceptance,absolute and unqualified,case example,{Acceptance:absolute and unqualified:case exam...,"[4.052, 4.053, 4.054, 4.055, 4.056, 4.057, 4.0...","[4.052, 4.053, 4.054, 4.055, 4.056, 4.057, 4.0..."
3,4,Acceptance,acceptor’s advantages,NaN,{Acceptance:acceptor’s advantages},"[4.052, 4.053, 4.054, 4.055, 4.056, 4.057, 4.0...","[4.052, 4.053, 4.054, 4.055, 4.056, 4.057, 4.0..."
4,5,Acceptance,characteristics,NaN,{Acceptance:characteristics},"[4.052, 4.053, 4.054, 4.055, 4.056, 4.057, 4.0...","[4.052, 4.053, 4.054, 4.055, 4.056, 4.057, 4.0..."
5,6,Acceptance,communication of,completion,{Acceptance:communication of:completion},"[4.052, 4.053, 4.054, 4.055, 4.056, 4.057, 4.0...","[4.052, 4.053, 4.054, 4.055, 4.056, 4.057, 4.0..."
6,7,Acceptance,communication of,conditions unfulfilled despite,{Acceptance:communication of:conditions unfulf...,"[4.052, 4.053, 4.054, 4.055, 4.056, 4.057, 4.0...","[4.052, 4.053, 4.054, 4.055, 4.056, 4.057, 4.0..."
...,...,...,...,...,...,...,...
1267,1268,Words and phrases,consideration,NaN,{Words and phrases:consideration},"[5.004, 5.005, 5.006, 5.007, 5.008, 5.009]","[5.004, 5.005, 5.006, 5.007, 5.008, 5.009]"
1280,1281,Words and phrases,invitation to treat,NaN,{Words and phrases:invitation to treat},[[4.024]],[4.024]
1284,1285,Words and phrases,offer,NaN,{Words and phrases:offer},[[4.014]],[4.014]
1293,1294,Words and phrases,sale by auction,NaN,{Words and phrases:sale by auction},[[4.034]],[4.034]


In [134]:

html_texts = []

for index, row in filtered_df.iterrows():
    id_count =  row['ID']
    primaryie_text = row['primaryie_text']
    secondaryies_text = row['secondaryies_text']
    tertiaryies_text = row['tertiaryies_text']
    paragraphs = row['page_nums_converted']
    merged_text = row['merged_text']
    
    if type(tertiaryies_text) == float:
        tertiaryies_text = ''
    htmlText = {
            'id': id_count,
            'primary': primaryie_text,
            'secondaryies':secondaryies_text ,
            'tertiaryies': tertiaryies_text,
            'combine_text': merged_text,
            'paragraphs':paragraphs
        }
    
    html_texts.append(htmlText)


In [135]:
for item in html_texts:
    print(item, ",")

{'id': 3, 'primary': 'Acceptance', 'secondaryies': 'absolute and unqualified', 'tertiaryies': 'case example', 'combine_text': '{Acceptance:absolute and unqualified:case example}', 'paragraphs': [4.052, 4.053, 4.054, 4.055, 4.056, 4.057, 4.058, 4.059, 4.06, 4.061, 4.062, 4.063, 4.054, 4.055]} ,
{'id': 4, 'primary': 'Acceptance', 'secondaryies': 'acceptor’s advantages', 'tertiaryies': '', 'combine_text': '{Acceptance:acceptor’s advantages}', 'paragraphs': [4.052, 4.053, 4.054, 4.055, 4.056, 4.057, 4.058, 4.059, 4.06, 4.061, 4.062, 4.063, 4.119]} ,
{'id': 5, 'primary': 'Acceptance', 'secondaryies': 'characteristics', 'tertiaryies': '', 'combine_text': '{Acceptance:characteristics}', 'paragraphs': [4.052, 4.053, 4.054, 4.055, 4.056, 4.057, 4.058, 4.059, 4.06, 4.061, 4.062, 4.063, 4.053]} ,
{'id': 6, 'primary': 'Acceptance', 'secondaryies': 'communication of', 'tertiaryies': 'completion', 'combine_text': '{Acceptance:communication of:completion}', 'paragraphs': [4.052, 4.053, 4.054, 4.055, 